In [1]:
#! Install library
import os
import importlib.util

if importlib.util.find_spec("requests") is None:
    os.system("pip install requests")

if importlib.util.find_spec("beautifulsoup4") is None:
    os.system("pip install beautifulsoup4")

if importlib.util.find_spec("Counter") is None:
    os.system("pip install Counter")

if importlib.util.find_spec("openpyxl") is None:
    os.system("pip install openpyxl")

In [2]:
Debug = False

In [3]:
#! Authentication details
import os
import dotenv
import importlib.util

JiraHost = None
ConfluenceHost = None
Username = None
Password = None

if importlib.util.find_spec("google.colab") is not None:  ## if using google colab
    if not os.path.exists(".env"):
        from google.colab import files

        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, ".env")
        except:
            pass

try:
    dotenv.load_dotenv("../.env", override=True)

    JiraHost = os.getenv("SECRETS_HOST")
    ConfluenceHost = os.getenv("SECRETS_CONFLUENCE")
    Username = os.getenv("SECRETS_USERNAME")
    Password = os.getenv("SECRETS_PASSWORD")
except:
    display("trouble loading dot env")
    pass

if JiraHost is None or JiraHost == "":
    JiraHost = input("Enter Jira Host")

if ConfluenceHost is None or ConfluenceHost == "":
    ConfluenceHost = input("Enter Confluence Host")

if Username is None or Username == "":
    Username = input("Enter Username")

if Password is None or Password == "":
    Password = input("Enter Password")

display("Jira Host: " + JiraHost)
display("Confluence Host: " + ConfluenceHost)

'Jira Host: https://jira.budgetdirect.com.au/'

'Confluence Host: https://confluence.budgetdirect.com.au/'

In [9]:
#! Functions
import base64
import re
from bs4 import BeautifulSoup
import requests
from collections import Counter
import pandas as pd
import requests
from functools import reduce
import warnings
import json

requests.packages.urllib3.disable_warnings(
    requests.packages.urllib3.exceptions.InsecureRequestWarning
)


def _ExpandColumn(
    self: pd.DataFrame,
    colName: str,
    columnsToExpand=[],
    prefix: str = "Prefix",
    sentenceCase: bool = True,
) -> pd.DataFrame:
    if prefix == "Prefix":
        prefix = colName + " "
        with warnings.catch_warnings():
            warnings.simplefilter(action="ignore", category=FutureWarning)
            expandedCols = self[colName].apply(
                lambda x: pd.Series(x).add_prefix(prefix)
            )
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))

    if len(columnsToExpand) > 0:
        expandedCols = expandedCols[columnsToExpand]

    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns]

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)


pd.DataFrame.expand = _ExpandColumn


def fnSentenceCase(s):
    s = " ".join(dict.fromkeys(s.split()))  # remove duplicate words
    s = s.replace("0", "")  # remove "0"
    s = s.strip()
    return " ".join(
        [x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]
    )  # sentence case


def _SentenceCaseColumns(self: pd.DataFrame) -> pd.DataFrame:
    self.columns = [fnSentenceCase(c) for c in self.columns]
    return self


pd.DataFrame.sentence_case_columns = _SentenceCaseColumns


def flatten_reduce_lambda(frm):
    try:
        return list(reduce(lambda x, y: x + y, frm, []))
    except:
        return list(reduce(lambda x, y: x + y, [frm], []))


def most_frequent(List):
    try:
        c = Counter(List)
        most_common = [key for key, _ in c.most_common(5)]
        return most_common
    except:
        return ""


def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic "
        + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120",
    }


def fnUpdateBoardAdmins(boardId, boardAdmins, boardGroups):
    boardAdminsData = {}
    boardAdminsData["userKeys"] = boardAdmins
    boardAdminsData["groupKeys"] = boardGroups
    data = {}
    data["id"] = boardId
    data["boardAdmins"] = boardAdminsData
    url = "/rest/greenhopper/1.0/rapidviewconfig/boardadmins"
    headers = fnGetDefaultHeaders()
    response = requests.put(
        JiraHost + url, headers=headers, data=json.dumps(data), verify=False
    )
    return response.json()


def fnUpdateFilterOwner(filterId, jql, name, description, filterOwner):
    data = {}
    data["jql"] = jql
    data["name"] = name
    data["description"] = description
    data["owner"] = {}
    data["owner"]["key"] = filterOwner
    url = "/rest/api/2/filter/" + str(filterId)
    headers = fnGetDefaultHeaders()
    response = requests.put(
        JiraHost + url, headers=headers, data=json.dumps(data), verify=False
    )
    return response.json()


def fnUpdateFilterOwnerAndShare(filterId, jql, name, description, filterOwner):
    permission = {}
    permission["type"] = "authenticated"
    permission["view"] = "true"
    # permission["edit"] = "true"

    # permission["type"] = "group"
    # permission["group"] = {}
    # permission["group"]["name"] = "jira-users"
    data = permission
    url = "/rest/api/2/filter/" + str(filterId) + "/permission"
    headers = fnGetDefaultHeaders()
    response = requests.post(
        JiraHost + url, headers=headers, data=json.dumps(data), verify=False
    )
    return response.json()


def fnGetFilterDetails(filterId):
    url = "/rest/api/2/filter/" + str(filterId)
    headers = fnGetDefaultHeaders()
    response = requests.get(JiraHost + url, headers=headers, verify=False)
    return response.json()


def fnBoards():
    def ApiCall(startAt):
        url = "/rest/greenhopper/1.0/rapidviews/viewsData"
        headers = fnGetDefaultHeaders()
        defaultContents = {"startAt": startAt}
        response = requests.get(
            JiraHost + url, headers=headers, json=defaultContents, verify=False
        )
        return response.json()

    values = fnAPI(ApiCall)
    return values


def fnUsers():
    # /rest/api/2/user/search?username=.&includeInactive=true&maxResults=1000&startAt=7000
    def ApiCall(startAt):
        url = (
            "/rest/api/2/user/search?username=.&includeInactive=true&maxResults=1000&startAt="
            + str(startAt)
        )
        headers = fnGetDefaultHeaders()
        response = requests.get(JiraHost + url, headers=headers, verify=False)
        return response.json()

    df = pd.DataFrame(flatten_reduce_lambda([ApiCall(i * 1000) for i in range(8)]))
    df = df.dropna(how="all")
    return df


def fnAPI(webRequestDelegate, startAt=0) -> pd.DataFrame:
    def innerGetResults(webRequestDelegate, startAt=0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(
                    webRequestDelegate, startAt + results["maxResults"]
                )
            else:
                return [results]
        else:
            return [results]

    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df

In [13]:
users = fnUsers()

display(users.head())

,self,key,name,emailAddress,avatarUrls,displayName,active,deleted,timeZone,locale
0,https://jira.budgetdirect.com.au/rest/api/2/us...,aadelakun,aadelakun,Ade.Adelakun@aihco.com.au,{'48x48': 'https://jira.budgetdirect.com.au/se...,Ade Adelakun,False,False,Australia/Brisbane,en_AU
1,https://jira.budgetdirect.com.au/rest/api/2/us...,JIRAUSER32069,aalang,aaron.lang@autogeneral.com.au,{'48x48': 'https://jira.budgetdirect.com.au/se...,Aaron Lang,True,False,Australia/Brisbane,en_AU
2,https://jira.budgetdirect.com.au/rest/api/2/us...,aalburydor,aalburydor,andrew.alburydor@autogeneral.com.au,{'48x48': 'https://jira.budgetdirect.com.au/se...,Andrew Albury-Dor [X],False,False,Australia/Brisbane,en_AU
3,https://jira.budgetdirect.com.au/rest/api/2/us...,JIRAUSER34432,aalkhanji,anas.alkhanji@autogeneral.com.au,{'48x48': 'https://jira.budgetdirect.com.au/se...,Anas Alkhanji,True,False,Australia/Brisbane,en_AU
4,https://jira.budgetdirect.com.au/rest/api/2/us...,JIRAUSER33748,aalmaradni,ahmad.almaradni@autogeneral.com.au,{'48x48': 'https://jira.budgetdirect.com.au/se...,Ahmad Almaradni,True,False,Australia/Brisbane,en_AU


In [25]:
duplicatedUsers = users[users.duplicated(["emailAddress"], keep=False)]

duplicatedUsers.to_excel("../ListDuplicatedUsers.xlsx", index=False)

display(duplicatedUsers)

,self,key,name,emailAddress,avatarUrls,displayName,active,deleted,timeZone,locale
47,https://jira.budgetdirect.com.au/rest/api/2/us...,achristian,achristian,alethea.christian@autogeneral.com.au,{'48x48': 'https://jira.budgetdirect.com.au/se...,Alethea Christian [X],False,False,Australia/Brisbane,en_AU
166,https://jira.budgetdirect.com.au/rest/api/2/us...,JIRAUSER32295,alchristia,alethea.christian@autogeneral.com.au,{'48x48': 'https://jira.budgetdirect.com.au/se...,Alethea Christian [X],False,False,Australia/Brisbane,en_AU
314,https://jira.budgetdirect.com.au/rest/api/2/us...,JIRAUSER33035,Ass-Developer,,{'48x48': 'https://jira.budgetdirect.com.au/se...,Graduate or Associate Developer,False,False,Australia/Brisbane,en_AU
315,https://jira.budgetdirect.com.au/rest/api/2/us...,JIRAUSER32719,assess04,,{'48x48': 'https://jira.budgetdirect.com.au/se...,04 - State Manager (Motor Assessing),False,False,Australia/Brisbane,en_AU
316,https://jira.budgetdirect.com.au/rest/api/2/us...,JIRAUSER32720,assess05,,{'48x48': 'https://jira.budgetdirect.com.au/se...,05 - General Manager Assessing,False,False,Australia/Brisbane,en_AU
...,...,...,...,...,...,...,...,...,...,...
3946,https://jira.budgetdirect.com.au/rest/api/2/us...,JIRAUSER32765,tpds02,,{'48x48': 'https://jira.budgetdirect.com.au/se...,02 - TPD Settlements - Team Leader,False,False,Australia/Brisbane,en_AU
3949,https://jira.budgetdirect.com.au/rest/api/2/us...,JIRAUSER32766,tpir02,,{'48x48': 'https://jira.budgetdirect.com.au/se...,02 - TPI Recoveries - Team Leader,False,False,Australia/Brisbane,en_AU
3950,https://jira.budgetdirect.com.au/rest/api/2/us...,JIRAUSER32767,tpis02,,{'48x48': 'https://jira.budgetdirect.com.au/se...,02 - TPI Settlements - Team Leader,False,False,Australia/Brisbane,en_AU
4047,https://jira.budgetdirect.com.au/rest/api/2/us...,JIRAUSER32437,Viviana,viviana.jaimes@autogeneral.com.au,{'48x48': 'https://jira.budgetdirect.com.au/se...,Viviana Jaimes,False,False,Australia/Brisbane,en_AU
